In [1]:
import pandas as pd
import numpy as np
import os
import zipfile
import json
from collections import Counter
from transformers import BertTokenizer

In [3]:
def read_zipped_json(filepath):
    with zipfile.ZipFile(filepath) as myzip:
        for filename in myzip.namelist():
            with myzip.open(filename) as myfile:
                return json.load(myfile)

In [4]:
t = read_zipped_json('./data/raw/val.json.zip')

In [2]:
tokenizer = BertTokenizer.from_pretrained("hfl/chinese-bert-wwm-ext")

Downloading:   0%|          | 0.00/110k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

In [17]:
utterance = '你好，我想找家人均消费在100-150元的餐馆吃驴杂汤这道菜，请给我推荐一家餐馆用餐吧。'
tokenizer.tokenize(utterance)
idx = utterance.index('餐馆')
len(tokenizer.tokenize(utterance[:idx]))

17

In [12]:
processed_data = []
all_intent = []
all_tag = []
context_size = 3
for item in t.values():
    context = []
    print(item)
    for i, turn in enumerate(item['messages']):
        utterance = turn["content"]
        tokens = tokenizer.tokenize(utterance)
#         tags = []
        tags = ['O'] * len(tokens)
        intents = []
        golden = []
        for intent, domain, slot, value in turn['dialog_act']:
            if intent in ['Inform', 'Recommend'] and '酒店设施' not in slot:
                if value in utterance:
                    idx = utterance.index(value)
                    start = len(tokenizer.tokenize(utterance[:idx]))
                    end = idx + len(tokenizer.tokenize(value))
                    tag = '+'.join((intent, domain, slot))
                    tags[start: end] = ['B+' + tag] + ['I+' + tag]* (end - start - 1)
                    token_v = ''.join(tokens[start: end]).replace('##', '')
                    golden.append([intent, domain, slot, token_v])
                else:
                    golden.append([intent, domain, slot, value])
            else:
                intents.append('+'.join([intent, domain, slot, value]))
                golden.append([intent, domain, slot, value])
        processed_data.append([tokens, tags, intents, golden, context[-context_size:]])
        all_intent.extend(intents)
        all_tag.extend(tags)
        context.append(turn['content'])
        
    break

{'sys-usr': [120, 20], 'goal': [[1, '餐馆', '推荐菜', ['驴杂汤'], False], [1, '餐馆', '人均消费', '100-150元', False], [1, '餐馆', '名称', '', False], [1, '餐馆', '周边景点', [], False], [1, '餐馆', '地址', '', False], [2, '景点', '评分', '4.5分以上', False], [2, '景点', '游玩时间', '3小时 - 4小时', False], [2, '景点', '名称', '', False], [2, '景点', '地址', '', False], [2, '景点', '周边景点', [], False], [3, '景点', '名称', '出现在id=2的周边景点里', False], [3, '景点', '门票', '免费', False], [3, '景点', '评分', '4.5分以上', False], [3, '景点', '周边餐馆', [], False], [4, '酒店', '价格', '400-500元', False], [4, '酒店', '酒店设施-所有房间提供wifi', '是', False], [4, '酒店', '酒店设施-早餐服务', '是', False], [4, '酒店', '名称', '', False], [4, '酒店', '酒店类型', '', False], [4, '酒店', '周边餐馆', [], False]], 'messages': [{'content': '你好，我想找家人均消费在100-150元的餐馆吃驴杂汤这道菜，请给我推荐一家餐馆用餐吧。', 'role': 'usr', 'dialog_act': [['General', 'greet', 'none', 'none'], ['Inform', '餐馆', '人均消费', '100-150元'], ['Inform', '餐馆', '推荐菜', '驴杂汤'], ['Request', '餐馆', '名称', '']], 'user_state': [[1, '餐馆', '推荐菜', ['驴杂汤'], True], [1, '餐馆', '人均消费', '100-1